In [2]:
import requests as req
from bs4 import BeautifulSoup as bs

from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time
from tqdm import tqdm

In [11]:
url = "https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010007&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EB%A7%A8%EC%A6%88%EC%BC%80%EC%96%B4"
driver = wb.Chrome()
driver.maximize_window()
driver.get(url)

In [10]:
brand_list = []        # 브랜드 명
item_name_list=[]      # 상품 명
category_list = []     # 카테고리 분류
price_list = []        # 정가
sale_price_list = []   # 할인가
stars_list = []        # 별점
type_list = []         # 주요 특징(?)
reviews_list = []      # 리뷰

for i in range(13,19):
    for j in range(1,5):
        rank = driver.find_element(By.CSS_SELECTOR, f"#Container > div.best-area > div.TabsConts.on > ul:nth-child({i}) > li:nth-child({j}) > div > a > img")
        rank.click()
        brand = driver.find_element(By.CSS_SELECTOR, "a.pd_arrow_link")
        brand_list.append(brand.text)
        item_name = driver.find_element(By.CSS_SELECTOR, "p.prd_name")
        item_name_list.append(item_name.text)
        category = driver.find_element(By.CSS_SELECTOR, "a#dtlCatNm")
        category_list.append(category.text)
        try : price = driver.find_element(By.CSS_SELECTOR, "span.price-1")
        except: price = driver.find_element(By.CSS_SELECTOR, "span.price-2")
        price_list.append(price.text.replace("\n",""))
        sale_price = driver.find_element(By.CSS_SELECTOR, "span.price-2")
        sale_price_list.append(sale_price.text.replace("\n",""))
        stars = driver.find_element(By.CSS_SELECTOR,  '#repReview > b') 
        stars_list.append(stars.text)

        # 페이지 이동 및 정렬 클릭
        body = driver.find_element(By.CSS_SELECTOR, 'body')
        body.send_keys(Keys.SPACE)
        time.sleep(1)
        
        review = driver.find_element(By.CSS_SELECTOR, '#reviewInfo > a')
        review.click()
        
        body = driver.find_element(By.CSS_SELECTOR, 'body')
        body.send_keys(Keys.SPACE)
        time.sleep(1)

        try:
            type = driver.find_elements(By.CSS_SELECTOR, '#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample') #"div.poll_sample")
            type_list.append(type[0].text.split())  
        except: pass
            
        helpful = driver.find_element(By.CSS_SELECTOR, "#gdasSort > li:nth-child(2) > a")
        helpful.click()
        
        body = driver.find_element(By.CSS_SELECTOR, 'body')
        body.send_keys(Keys.SPACE)
        time.sleep(1)
        
        # 첫 페이지 리뷰 수집 함수
        def collect_reviews():
            for i in range(1, 11):
                try:
                    review = driver.find_element(By.CSS_SELECTOR, f"#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner")
                    reviews_list.append(review.text)
                except Exception as e:
                    print(f"{i}번째 리뷰를 찾을 수 없습니다🤖: {e}")
                time.sleep(1)
        
        # 첫 페이지 리뷰 수집
        collect_reviews()
        
        # 2~10 페이지 리뷰 수집
        for j in range(2, 11):
            try:
                page = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({j})')
                page.click()
                time.sleep(2)
                collect_reviews()
            except Exception as e:
                print(f"{j}번째 페이지를 클릭할 수 없습니다🤖: {e}")
        
        max_iterations = 10  # 최대 반복 횟수
        iteration = 0  # 현재 반복 횟수 초기화
        
        while iteration < max_iterations:
            try:
                # '다음' 버튼 찾기
                next_button = driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next')
                next_button.click()  # '다음' 버튼 클릭
                time.sleep(2)  # 페이지 로딩 대기
        
                # 페이지 버튼 클릭 및 리뷰 수집
                for k in range(3, 12):
                    try:
                        page_button = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({k})')
                        page_button.click()  # 페이지 버튼 클릭
                        time.sleep(2)  # 페이지 로딩 대기
                        collect_reviews()
                    except Exception as e:
                        print(f"{k}번째 페이지 버튼 클릭 실패🤖: {e}")
                        break  # 버튼 클릭 실패 시 반복문 탈출
                iteration += 1  # 반복 횟수 증가
            except Exception as e:
                print(f"no more page🍒: {e}")
                break  # '다음' 버튼이 없으면 반복문 탈출
                
               
        driver.back()
        time.sleep(2)

3번째 페이지 버튼 클릭 실패🤖: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#gdasContentsArea > div > div.pageing > a:nth-child(3)"}
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF776C1EEA2+31554]
	(No symbol) [0x00007FF776B97ED9]
	(No symbol) [0x00007FF776A5872A]
	(No symbol) [0x00007FF776AA8434]
	(No symbol) [0x00007FF776AA853C]
	(No symbol) [0x00007FF776AEF6A7]
	(No symbol) [0x00007FF776ACD06F]
	(No symbol) [0x00007FF776AEC977]
	(No symbol) [0x00007FF776ACCDD3]
	(No symbol) [0x00007FF776A9A33B]
	(No symbol) [0x00007FF776A9AED1]
	GetHandleVerifier [0x00007FF776F28B1D+3217341]
	GetHandleVerifier [0x00007FF776F75AE3+3532675]
	GetHandleVerifier [0x00007FF776F6B0E0+3489152]
	GetHandleVerifier [0x00007FF776CCE776+750614]
	(No symbol) [0x00007FF776BA375F]
	(No symbol) [0x00007FF776

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF776C1EEA2+31554]
	(No symbol) [0x00007FF776B97ED9]
	(No symbol) [0x00007FF776A5872A]
	(No symbol) [0x00007FF776A2D995]
	(No symbol) [0x00007FF776AD44D7]
	(No symbol) [0x00007FF776AEC051]
	(No symbol) [0x00007FF776ACCDD3]
	(No symbol) [0x00007FF776A9A33B]
	(No symbol) [0x00007FF776A9AED1]
	GetHandleVerifier [0x00007FF776F28B1D+3217341]
	GetHandleVerifier [0x00007FF776F75AE3+3532675]
	GetHandleVerifier [0x00007FF776F6B0E0+3489152]
	GetHandleVerifier [0x00007FF776CCE776+750614]
	(No symbol) [0x00007FF776BA375F]
	(No symbol) [0x00007FF776B9EB14]
	(No symbol) [0x00007FF776B9ECA2]
	(No symbol) [0x00007FF776B8E16F]
	BaseThreadInitThunk [0x00007FFB3B26257D+29]
	RtlUserThreadStart [0x00007FFB3B5EAF28+40]


In [ ]:
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 초기 빈 리스트 생성
brand_list = []        # 브랜드 명
item_name_list = []    # 상품 명
category_list = []     # 카테고리 분류
price_list = []        # 정가
sale_price_list = []   # 할인가
stars_list = []        # 별점
type_list = []         # 주요 특징(?)
reviews_list = []      # 리뷰

# CSV 파일의 컬럼 이름 정의
columns = ['브랜드', '상품', '카테고리', '정가', '할인가', '별점', '주요 특징', '리뷰']

# 빈 데이터프레임 생성 및 CSV 파일로 저장 (헤더 포함)
df_empty = pd.DataFrame(columns=columns)
df_empty.to_csv('products.csv', index=False, encoding='utf-8-sig')

# 데이터프레임을 CSV 파일에 추가하는 함수
def append_to_csv(data):
    df = pd.DataFrame(data, columns=columns)
    df.to_csv('products.csv', mode='a', index=False, header=False, encoding='utf-8-sig')

# 첫 페이지 리뷰 수집 함수
def collect_reviews():
    reviews = []
    for i in range(1, 11):
        try:
            review = driver.find_element(By.CSS_SELECTOR, f"#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner")
            reviews.append(review.text)
        except Exception as e:
            print(f"{i}번째 리뷰를 찾을 수 없습니다🤖: {e}")
        time.sleep(1)
    return reviews

for i in range(1, 26):
    for j in range(1, 5):
        rank = driver.find_element(By.CSS_SELECTOR, f"#Container > div.best-area > div.TabsConts.on > ul:nth-child({i}) > li:nth-child({j}) > div > a > img")
        rank.click()
        brand = driver.find_element(By.CSS_SELECTOR, "a.pd_arrow_link")
        brand_list.append(brand.text)
        item_name = driver.find_element(By.CSS_SELECTOR, "p.prd_name")
        item_name_list.append(item_name.text)
        category = driver.find_element(By.CSS_SELECTOR, "a#dtlCatNm")
        category_list.append(category.text)
        try:
            price = driver.find_element(By.CSS_SELECTOR, "span.price-1")
        except:
            price = driver.find_element(By.CSS_SELECTOR, "span.price-2")
        price_list.append(price.text.replace("\n", ""))
        sale_price = driver.find_element(By.CSS_SELECTOR, "span.price-2")
        sale_price_list.append(sale_price.text.replace("\n", ""))
        stars = driver.find_element(By.CSS_SELECTOR, '#repReview > b')
        stars_list.append(stars.text)

        # 페이지 이동 및 정렬 클릭
        body = driver.find_element(By.CSS_SELECTOR, 'body')
        body.send_keys(Keys.SPACE)
        time.sleep(1)

        review = driver.find_element(By.CSS_SELECTOR, '#reviewInfo > a')
        review.click()

        body = driver.find_element(By.CSS_SELECTOR, 'body')
        body.send_keys(Keys.SPACE)
        time.sleep(1)

        try:
            type = driver.find_elements(By.CSS_SELECTOR, '#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample')
            type_list.append(type[0].text.split())
        except:
            type_list.append('')

        helpful = driver.find_element(By.CSS_SELECTOR, "#gdasSort > li:nth-child(2) > a")
        helpful.click()

        body = driver.find_element(By.CSS_SELECTOR, 'body')
        body.send_keys(Keys.SPACE)
        time.sleep(1)

        reviews = collect_reviews()

        # 2~10 페이지 리뷰 수집
        for page_num in range(2, 11):
            try:
                page = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({page_num})')
                page.click()
                time.sleep(2)
                reviews += collect_reviews()
            except Exception as e:
                print(f"{page_num}번째 페이지를 클릭할 수 없습니다🤖: {e}")

        max_iterations = 10  # 최대 반복 횟수
        iteration = 0  # 현재 반복 횟수 초기화

        while iteration < max_iterations:
            try:
                # '다음' 버튼 찾기
                next_button = driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next')
                next_button.click()  # '다음' 버튼 클릭
                time.sleep(2)  # 페이지 로딩 대기

                # 페이지 버튼 클릭 및 리뷰 수집
                for k in range(3, 12):
                    try:
                        page_button = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({k})')
                        page_button.click()  # 페이지 버튼 클릭
                        time.sleep(2)  # 페이지 로딩 대기
                        reviews += collect_reviews()
                    except Exception as e:
                        print(f"{k}번째 페이지 버튼 클릭 실패🤖: {e}")
                        break  # 버튼 클릭 실패 시 반복문 탈출
                iteration += 1  # 반복 횟수 증가
            except Exception as e:
                print(f"no more page🍒: {e}")
                break  # '다음' 버튼이 없으면 반복문 탈출

        # 수집된 데이터를 데이터프레임으로 변환 후 CSV 파일에 추가
        data = {
            'Brand': [brand.text] * len(reviews),
            'Item Name': [item_name.text] * len(reviews),
            'Category': [category.text] * len(reviews),
            'Price': [price.text.replace("\n", "")] * len(reviews),
            'Sale Price': [sale_price.text.replace("\n", "")] * len(reviews),
            'Stars': [stars.text] * len(reviews),
            'Type': [type[0].text.split() if type else ''] * len(reviews),
            'Reviews': reviews
        }
        append_to_csv(data)

        driver.back()
        time.sleep(2)


In [19]:
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 초기 빈 리스트 생성
brand_list = []        # 브랜드 명
item_name_list = []    # 상품 명
category_list = []     # 카테고리 분류
price_list = []        # 정가
sale_price_list = []   # 할인가
stars_list = []        # 별점
type_list = []         # 주요 특징(?)
reviews_list = []      # 리뷰

# CSV 파일의 컬럼 이름 정의
columns = ['브랜드', '상품명', '카테고리', '정가', '할인가', '별점', '주요 특징', '리뷰']

# 빈 데이터프레임 생성 및 CSV 파일로 저장 (헤더 포함)
df_empty = pd.DataFrame(columns=columns)
df_empty.to_csv('C:/Users/un991/새싹2기/project_final/올영.csv', index=False, encoding='utf-8-sig')

# 데이터프레임을 CSV 파일에 추가하는 함수
def append_to_csv(data):
    df = pd.DataFrame(data, columns=columns)
    df.to_csv('C:/Users/un991/새싹2기/project_final/올영.csv', mode='a', index=False, header=False, encoding='utf-8-sig')

# 첫 페이지 리뷰 수집 함수
def collect_reviews():
    reviews = []
    for i in range(1, 11):
        try:
            review = driver.find_element(By.CSS_SELECTOR, f"#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner")
            reviews.append(review.text)
        except Exception as e:
            print(f"{i}번째 리뷰를 찾을 수 없습니다🤖: {e}")
        time.sleep(1)
    return reviews


rank = driver.find_element(By.CSS_SELECTOR, "#Container > div.best-area > div.TabsConts.on > ul:nth-child(13) > li:nth-child(1) > div > a > img")
rank.click()
brand = driver.find_element(By.CSS_SELECTOR, "a.pd_arrow_link")
brand_list.append(brand.text)
item_name = driver.find_element(By.CSS_SELECTOR, "p.prd_name")
item_name_list.append(item_name.text)
category = driver.find_element(By.CSS_SELECTOR, "a#dtlCatNm")
category_list.append(category.text)
try:
    price = driver.find_element(By.CSS_SELECTOR, "span.price-1")
except:
    price = driver.find_element(By.CSS_SELECTOR, "span.price-2")
price_list.append(price.text.replace("\n", ""))
sale_price = driver.find_element(By.CSS_SELECTOR, "span.price-2")
sale_price_list.append(sale_price.text.replace("\n", ""))
stars = driver.find_element(By.CSS_SELECTOR, '#repReview > b')
stars_list.append(stars.text)

# 페이지 이동 및 정렬 클릭
body = driver.find_element(By.CSS_SELECTOR, 'body')
body.send_keys(Keys.SPACE)
time.sleep(1)

review = driver.find_element(By.CSS_SELECTOR, '#reviewInfo > a')
review.click()

body = driver.find_element(By.CSS_SELECTOR, 'body')
body.send_keys(Keys.SPACE)
time.sleep(1)

try:
    type = driver.find_elements(By.CSS_SELECTOR, '#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample')
    type_list.append(type[0].text.split())
except:
    type_list.append('')

helpful = driver.find_element(By.CSS_SELECTOR, "#gdasSort > li:nth-child(2) > a")
helpful.click()

body = driver.find_element(By.CSS_SELECTOR, 'body')
body.send_keys(Keys.SPACE)
time.sleep(1)

reviews = collect_reviews()

# 2~10 페이지 리뷰 수집
for page_num in range(2, 11):
    try:
        page = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({page_num})')
        page.click()
        time.sleep(2)
        reviews += collect_reviews()
    except Exception as e:
        print(f"{page_num}번째 페이지를 클릭할 수 없습니다🤖: {e}")

max_iterations = 10  # 최대 반복 횟수
iteration = 0  # 현재 반복 횟수 초기화

while iteration < max_iterations:
    try:
    # '다음' 버튼 찾기
        next_button = driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next')
        next_button.click()  # '다음' 버튼 클릭
        time.sleep(2)  # 페이지 로딩 대기

        # 페이지 버튼 클릭 및 리뷰 수집
        for k in range(3, 12):
            try:
                page_button = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({k})')
                page_button.click()  # 페이지 버튼 클릭
                time.sleep(2)  # 페이지 로딩 대기
                reviews += collect_reviews()
            except Exception as e:
                print(f"{k}번째 페이지 버튼 클릭 실패🤖: {e}")
                break  # 버튼 클릭 실패 시 반복문 탈출
        iteration += 1  # 반복 횟수 증가
    except Exception as e:
        print(f"no more page🍒: {e}")
        break  # '다음' 버튼이 없으면 반복문 탈출

# 수집된 데이터를 데이터프레임으로 변환 후 CSV 파일에 추가
data = {'브랜드': brand.text,
        '상품명': item_name.text,
        '카테고리': category.text,
        '정가': price.text.replace("\n", ""),
        '할인가': sale_price.text.replace("\n", ""),
        '별점': stars.text,
        '주요 특징': type[0].text.split() if type else '',
        '리뷰': reviews_list
        }
append_to_csv(data)

driver.back()
time.sleep(2)


3번째 리뷰를 찾을 수 없습니다🤖: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#gdasList > li:nth-child(3) > div.review_cont > div.txt_inner"}
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF776C1EEA2+31554]
	(No symbol) [0x00007FF776B97ED9]
	(No symbol) [0x00007FF776A5872A]
	(No symbol) [0x00007FF776AA8434]
	(No symbol) [0x00007FF776AA853C]
	(No symbol) [0x00007FF776AEF6A7]
	(No symbol) [0x00007FF776ACD06F]
	(No symbol) [0x00007FF776AEC977]
	(No symbol) [0x00007FF776ACCDD3]
	(No symbol) [0x00007FF776A9A33B]
	(No symbol) [0x00007FF776A9AED1]
	GetHandleVerifier [0x00007FF776F28B1D+3217341]
	GetHandleVerifier [0x00007FF776F75AE3+3532675]
	GetHandleVerifier [0x00007FF776F6B0E0+3489152]
	GetHandleVerifier [0x00007FF776CCE776+750614]
	(No symbol) [0x00007FF776BA375F]
	(No symbol) [0x00

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF776C1EEA2+31554]
	(No symbol) [0x00007FF776B97ED9]
	(No symbol) [0x00007FF776A5872A]
	(No symbol) [0x00007FF776A5E55E]
	(No symbol) [0x00007FF776A60EB1]
	(No symbol) [0x00007FF776A60F50]
	(No symbol) [0x00007FF776AA21DB]
	(No symbol) [0x00007FF776ACD02A]
	(No symbol) [0x00007FF776A9BA76]
	(No symbol) [0x00007FF776ACD240]
	(No symbol) [0x00007FF776AEC977]
	(No symbol) [0x00007FF776ACCDD3]
	(No symbol) [0x00007FF776A9A33B]
	(No symbol) [0x00007FF776A9AED1]
	GetHandleVerifier [0x00007FF776F28B1D+3217341]
	GetHandleVerifier [0x00007FF776F75AE3+3532675]
	GetHandleVerifier [0x00007FF776F6B0E0+3489152]
	GetHandleVerifier [0x00007FF776CCE776+750614]
	(No

In [18]:
len(reviews)

2